In [1]:
import pandas as pd
import warnings
from sklearn.linear_model import Ridge

In [2]:
air_quality_dat = pd.read_csv("AIR_QUALITY_DATA.csv")
air_quality_dat = air_quality_dat[air_quality_dat["Year"] == 2010] # TEMPORARY

paris_agreement_dat = pd.read_csv("PARIS_AGREEMENT_DATA.csv")

greenhouse_gas_dat = pd.read_csv("GHG_DATA.csv")
greenhouse_gas_dat = greenhouse_gas_dat[greenhouse_gas_dat["Year"] == 1973] # TEMPORARY

In [3]:
greenhouse_gas_dat

Unnamed: 0  Year                                            Country  \
530         531  1973  Aruba                                         ...   
531         532  1973  Afghanistan                                   ...   
532         533  1973  Angola                                        ...   
533         534  1973  Anguilla                                      ...   
534         535  1973  Albania                                       ...   
..          ...   ...                                                ...   
826         827  1973  Samoa                                         ...   
827         828  1973  Yemen                                         ...   
828         829  1973  South Africa                                  ...   
829         830  1973  Zambia                                        ...   
830         831  1973  Zimbabwe                                      ...   

     GHG_Emissions  
530       1.093028  
531       1.327430  
532       3.349278  
533       0.640851  
534       3.744120  
..             ...  
826       1.275889  
827       1.051631  
828      10.878933  
829       3.053328  
830       4.215143  

[208 rows x 4 columns]

In [4]:
air_quality_dat.head()

ID  Year                                          City_Name  \
7   15070  2010  Laitila                                       ...   
8   15071  2010  Lappeenranta                                  ...   
16  15079  2010  Lohja                                         ...   
24  15087  2010  Mikkeli                                       ...   
27  15090  2010  Muonio                                        ...   

                                         Country_Name    NO2  PM10  PM25  \
7   Finland                                       ...   1.30   NaN   NaN   
8   Finland                                       ...  13.00  18.0   9.3   
16  Finland                                       ...    NaN  12.0   7.2   
24  Finland                                       ...    NaN  14.0   NaN   
27  Finland                                       ...   0.64   NaN   NaN   

    NO2_TEMPORAL_COVERAGE  PM25_TEMPORAL_COVERAGE  PM10_TEMPORAL_COVERAGE  
7                    95.0                     NaN                     NaN  
8                    98.0                    98.0                    99.0  
16                    NaN                    97.0                    97.0  
24                    NaN                     NaN                   100.0  
27                   98.0                     NaN                     NaN

In [5]:
merged_data = air_quality_dat.merge(paris_agreement_dat, how = "left", on = "Country_Name").merge(greenhouse_gas_dat, how = "left", left_on = ["Country_Name"], right_on = ["Country"])
merged_data.head()

ID  Year_x                                          City_Name  \
0  15070    2010  Laitila                                       ...   
1  15071    2010  Lappeenranta                                  ...   
2  15079    2010  Lohja                                         ...   
3  15087    2010  Mikkeli                                       ...   
4  15090    2010  Muonio                                        ...   

                                        Country_Name    NO2  PM10  PM25  \
0  Finland                                       ...   1.30   NaN   NaN   
1  Finland                                       ...  13.00  18.0   9.3   
2  Finland                                       ...    NaN  12.0   7.2   
3  Finland                                       ...    NaN  14.0   NaN   
4  Finland                                       ...   0.64   NaN   NaN   

   NO2_TEMPORAL_COVERAGE  PM25_TEMPORAL_COVERAGE  PM10_TEMPORAL_COVERAGE  \
0                   95.0                     NaN                     NaN   
1                   98.0                    98.0                    99.0   
2                    NaN                    97.0                    97.0   
3                    NaN                     NaN                   100.0   
4                   98.0                     NaN                     NaN   

   Unnamed: 0_x Date_Joined  Unnamed: 0_y  Year_y  \
0            60  2016-11-14         593.0  1973.0   
1            60  2016-11-14         593.0  1973.0   
2            60  2016-11-14         593.0  1973.0   
3            60  2016-11-14         593.0  1973.0   
4            60  2016-11-14         593.0  1973.0   

                                             Country  GHG_Emissions  
0  Finland                                       ...      16.037502  
1  Finland                                       ...      16.037502  
2  Finland                                       ...      16.037502  
3  Finland                                       ...      16.037502  
4  Finland                                       ...      16.037502

In [6]:
simplified_data = merged_data[["Country_Name", "NO2", "GHG_Emissions", "Unnamed: 0_x"]]

In [7]:
simplified_data = simplified_data.rename(columns={"Unnamed: 0_x":"Paris_Index"})

In [8]:
warnings.filterwarnings('ignore')
simplified_data["Paris_Index"].loc[~simplified_data["Paris_Index"].isnull()] = 1
simplified_data["Paris_Index"].loc[simplified_data["Paris_Index"].isnull()] = 0
simplified_data.tail()

Country_Name   NO2  GHG_Emissions  \
225  France                                        ...  14.0            NaN   
226  France                                        ...  18.0            NaN   
227  France                                        ...   NaN            NaN   
228  France                                        ...  90.0            NaN   
229  France                                        ...  12.0            NaN   

     Paris_Index  
225            1  
226            1  
227            1  
228            1  
229            1

In [9]:
aggregated_data = simplified_data.groupby(['Country_Name',"GHG_Emissions","Paris_Index"]).sum().reset_index()
aggregated_data

Country_Name  GHG_Emissions  \
0  Finland                                       ...      16.037502   

   Paris_Index     NO2  
0            1  156.64

In [10]:
X = aggregated_data[["GHG_Emissions", "Paris_Index"]]
y = aggregated_data[["NO2"]]
ridge = Ridge(alpha=1.0, random_state=123)
ridge.fit(X, y)
coef_ridge = ridge.coef_
coef_ridge

array([[0., 0.]])